# Задача 5 (бонусная): экспериментальное исследование алгоритмов для регулярных запросов

## Подготовка окружения

Для исполнения кода, приведённого в данном исследовании, необходима система с ядром Linux, подготовленная при помощи выполнения команд, приведённых далее.

In [ ]:
# @formatter:off
!pip install -r ../requirements.txt
!pip install pycubool
# @formatter:on

## Введение

В данном работе рассматривается задача достижимости в графе с регулярными ограничениями и исследуется производительность её решений матричными подходами на языке Python при помощи библиотек, осуществляющих матричные операции на CPU и GPU.

### О задаче достижимости

Пусть дан конечный ориентированный помеченный граф $G = (V, E, L)$, в котором каждому пути $\pi \in E^*$, состоящему из рёбер с метками $l_1, ..., l_n \in L$, сопоставляется слово по правилу $\omega(\pi) = l_1 * ... * l_n$, где $*$ --- конкатенация.

Пусть также дано регулярное выражение $R$ для языка над алфавитом $L$.

В данных условиях мы будем рассматривать три варианта задачи о достижимости в графе $G$ с регулярными ограничениями, заданным $R$.

#### Постановка 1. Достижимость между всеми парами вершин.

Данная постановка задачи является наиболее общей.

В ней задача заключается в нахождении таких пар вершин $G$, что между ними существует путь, которому сопоставляется слово из языка, задаваемого $R$. То есть задача заключается в нахождении множества $\{(v_i, v_j) \in V^2: \exists \pi \in E^*: start(\pi) = v_i, final(\pi) = v_j, \omega(\pi) \in language(R)\}$.

#### Постановка 2. Достижимость для всего множества заданных вершин.

В данной постановке задачи из множества $V$ всех вершин графа дополнительно выделяются подмножества стартовых и финальных вершин: $V_S \subseteq V$ и $V_F \subseteq V$ соответственно. Задача заключается в нахождении всех таких финальных вершин, таких, что до них существуют пути из стартовых, которым сопоставляются слова из языка, задаваемого $R$. То есть задача заключается в нахождении множества $\{v_i \in V_F: \exists \pi \in E^*: start(\pi) \in V_S, final(\pi) = v_j, \omega(\pi) \in language(R)\}$.

#### Постановка 3. Достижимость для каждой вершины из заданного множества стартовых вершин.

Данная постановка является комбинацией предыдущих двух. Здесь также выделяются множества стартовых и финальных вершин $V_S и $V_F, в которых найденные пути должны начинаться и оканчиваться соответственно, однако, аналогично постановке 1, требуется найти множество пар вершин-начал и вершин-концов данных путей. То есть задача заключается в нахождении множества $\{(v_i, v_j) \in V_S \times V_F: \exists \pi \in E^*: start(\pi) = v_i, final(\pi) = v_j, \omega(\pi) \in language(R)\}$.

### Исследуемые решения

В данной работе для решения поставленных выше задач используются методы, кратко описываемые следующим образом:

1. **Достижимость между всеми парами вершин**. Через тензорное произведение строится пересечение конечных автоматов, соответствующих графу и регулярному выражению, для которого затем считается транзитивное замыкание через матричное возведение в степень.
2. **Достижимость для всего множества заданных вершин**. Алгоритм решения основывается на синхронном обходе в ширину конечных автоматов, соответствующих регулярному выражению и графу. Обход проводится через умножение фронта, представленного как матрица условно разделенная на соответствующие две части, на матрицу смежности, построенную как прямая сумма матриц смежности обоих автоматов. Матрица фронта в левой части представляет из себя единичную матрицу, а в правой --- строки-фронты-обхода, возможные для соответствующей вершины в левой части. После каждого умножения матрица фронта трансформируется для поддержки данного инварианта.
3. **Достижимость для каждой вершины из заданного множества стартовых вершин**. Принцип решения схож с предыдущим пунктом, однако фронт разделяется для каждой стартовой вершины и представляется как единая матрица, что позволяет хранить информацию о стартовой вершине для каждой достигнутой финальной.

На практике матрицы смежности, используемые в представленных алгоритмах являются сильно разреженными, поэтому при их реализации становится выгодно представлять такие матрицы специальным образом, избегая хранения "пустых" ячеек. Каждый из описанных алгоритмов был реализован на языке Python при помощи двух библиотек, содержащих таких типы матриц:

1. [**scipy.sparse**](https://docs.scipy.org/doc/scipy/reference/sparse.html) --- модуль SciPy, предоставляющий несколько видов двухмерных разреженных массивов с интерфейсом взаимодействия, приближенным к таковому в NumPy. Все операции, проводимые с массивами, происходят на CPU.
2. [**pycubool**](https://github.com/JetBrains-Research/cuBool) --- обертка для C-библиотеки, позволяющая проводить операции линейной алгебры над разреженными матрицами на GPU марки NVIDIA при помощи технологии CUDA.

Оба способа реализации максимально идентичны (на сколько это позволяют интерфейсы представленных библиотек).

#### Реализация на scipy.sparse

Реализация алгоритмов, основанная на `scipy.sparse`, расположена в модуле [`project.rpq`](../project/rpq.py) данного репозитория.

In [ ]:
from project.rpq import BfsMode
from project.rpq import rpq_by_bfs
from project.rpq import rpq_by_tensor

#### Реализация на pycubool

Реализация алгоритмов, основанная на `pycubool`, представленная ниже:

In [ ]:
from itertools import product
from typing import Any
from typing import NamedTuple

import pycubool as cb
from pyformlang.finite_automaton import EpsilonNFA


class BoolDecompositionCuda:
    class StateInfo(NamedTuple):
        data: Any
        is_start: bool
        is_final: bool

    def __init__(
        self,
        states: list[StateInfo] | None = None,
        adjs: dict[Any, cb.Matrix] | None = None,
    ):
        self.states: list[BoolDecompositionCuda.StateInfo] = (
            states if states is not None else []
        )
        self.adjs: dict[Any, cb.Matrix] = adjs if adjs is not None else {}

    @classmethod
    def from_nfa(
        cls, nfa: EpsilonNFA, sort_states: bool = False
    ) -> "BoolDecompositionCuda":
        # Construct states, removing duplicates
        states = list(
            set(
                cls.StateInfo(
                    data=st.value,
                    is_start=st in nfa.start_states,
                    is_final=st in nfa.final_states,
                )
                for st in nfa.states
            )
        )
        if sort_states:
            states = sorted(states, key=lambda st: st.data)

        # Construct adjacency matrices
        adjs = {}
        transitions = nfa.to_dict()
        n = len(states)
        for n_from in transitions:
            for symbol, ns_to in transitions[n_from].items():
                adj = adjs.setdefault(symbol.value, cb.Matrix.empty((n, n)))
                beg_index = next(i for i, s in enumerate(states) if s.data == n_from)
                for n_to in ns_to if isinstance(ns_to, set) else {ns_to}:
                    end_index = next(i for i, s in enumerate(states) if s.data == n_to)
                    adj[beg_index, end_index] = True

        return cls(states, adjs)

    def intersect(self, other: "BoolDecompositionCuda") -> "BoolDecompositionCuda":
        # Set of states of the intersection is the product of the given sets of states
        # State is "start" if both of its element states are "start"
        # State is "final" if both of its element states are "final"
        states = [
            self.StateInfo(
                data=(st1.data, st2.data),
                is_start=st1.is_start and st2.is_start,
                is_final=st1.is_final and st2.is_final,
            )
            for st1, st2 in product(self.states, other.states)
        ]

        # Set of symbols of the intersection is the union of the given sets of symbols
        # Adjacency matrix for a symbol is a kronecker product of the given matrices of
        # this symbol
        adjs = {}
        n = len(states)
        for symbol in set(self.adjs.keys()).union(set(other.adjs.keys())):
            if symbol in self.adjs and symbol in other.adjs:
                adjs[symbol] = self.adjs[symbol].kronecker(other.adjs[symbol])
            else:
                adjs[symbol] = cb.Matrix.empty((n, n))

        return BoolDecompositionCuda(states, adjs)

    def transitive_closure_any_symbol(self) -> tuple[list[int], list[int]]:
        # Gather all matrices to get all existing paths
        n = len(self.states)
        adj_all = cb.Matrix.empty((n, n))
        for adj in self.adjs.values():
            adj_all = adj_all.ewiseadd(adj)

        # Transitive closure by repeated-squaring-like approach
        while True:
            # nvals gives the number of paths
            prev_path_num = adj_all.nvals
            # Multiplication gives new paths, while sum retains the old ones
            adj_all.mxm(adj_all, out=adj_all, accumulate=True)
            # If no new paths appear, all paths have been discovered
            if prev_path_num == adj_all.nvals:
                break

        # Convert to a more user-friendly representation
        return adj_all.to_lists()

    def _direct_sum(self, other: "BoolDecompositionCuda") -> "BoolDecompositionCuda":
        states = self.states + other.states

        adjs = {}
        for symbol in set(self.adjs.keys()).intersection(set(other.adjs.keys())):
            dsum = cb.Matrix.empty((len(states), len(states)))
            for i, j in self.adjs[symbol]:
                dsum[i, j] = True
            for i, j in other.adjs[symbol]:
                dsum[len(self.states) + i, len(self.states) + j] = True
            adjs[symbol] = dsum

        return BoolDecompositionCuda(states, adjs)

    def constrained_bfs(
        self, constraint: "BoolDecompositionCuda", separated: bool = False
    ) -> set[int] | set[tuple[int, int]]:
        # Save states number because will use them heavily for matrix construction
        n = len(constraint.states)
        m = len(self.states)

        direct_sum = constraint._direct_sum(self)

        # Create initial front from starts of constraint (left) and self (right)
        start_states_indices = [i for i, st in enumerate(self.states) if st.is_start]
        init_front = (
            _init_bfs_front(self.states, constraint.states)
            if not separated
            else _init_separated_bfs_front(
                self.states, constraint.states, start_states_indices
            )
        )

        # Create visited, fill with zeroes instead of init_front to get rid of initial
        # positions in the result
        visited = cb.Matrix.empty(init_front.shape)

        # Perform matrix-multiplication-based-BFS until visited stops changing
        while True:
            old_visited_nvals = visited.nvals

            # Perform a BFS step for each matrix in direct sum
            for _, adj in direct_sum.adjs.items():
                # Compute new front for the symbol
                front_part = (
                    visited.mxm(adj) if init_front is None else init_front.mxm(adj)
                )
                # Transform the resulting front so that:
                # 1. It only contains rows with non-zeros in both parts.
                # 2. Its left part contains non-zeroes only on its main diagonal.
                visited = visited.ewiseadd(_transform_front_part(front_part, n))

            # Can use visited instead now
            init_front = None

            # If no new non-zero elements have appeared, we've visited all we can
            if visited.nvals == old_visited_nvals:
                break

        # If visited a final self-state in final constraint-state, we found a result
        results = set()
        for i, j in visited.to_list():
            # Check that the element is from the self part (which is the main BFS part)
            # and the final state requirements are satisfied
            if j >= n and constraint.states[i % n].is_final:  # % is for separated BFS
                self_st_index = j - n
                if self.states[self_st_index].is_final:
                    results.add(
                        self_st_index
                        if not separated
                        else (start_states_indices[i // n], self_st_index)
                    )
        return results


def _init_bfs_front(
    self_states: list[BoolDecompositionCuda.StateInfo],
    constr_states: list[BoolDecompositionCuda.StateInfo],
    self_start_indices: list[int] | None = None,
) -> cb.Matrix:
    front = cb.Matrix.empty((len(constr_states), len(constr_states) + len(self_states)))

    if self_start_indices is None:
        self_start_indices = [j for j, st in enumerate(self_states) if st.is_start]

    for i, st in enumerate(constr_states):
        if st.is_start:
            front[i, i] = True  # Mark diagonal element as start
            for j in self_start_indices:
                front[i, len(constr_states) + j] = True  # Fill start row

    return front


def _init_separated_bfs_front(
    self_states: list[BoolDecompositionCuda.StateInfo],
    constr_states: list[BoolDecompositionCuda.StateInfo],
    start_states_indices: list[int],
) -> cb.Matrix:
    fronts = [
        _init_bfs_front(
            self_states,
            constr_states,
            self_start_indices=[st_i],
        )
        for st_i in start_states_indices
    ]

    if len(fronts) == 0:
        return cb.Matrix.empty(
            (len(constr_states), len(constr_states) + len(self_states))
        )

    # Build the united front as a vertical stack of the separated fronts
    result = cb.Matrix.empty(
        (len(fronts) * len(constr_states), len(constr_states) + len(self_states))
    )
    vstack_helper = cb.Matrix.empty((len(fronts), 1))
    for i, front in enumerate(fronts):
        vstack_helper.build(rows={i}, cols={0})
        result = result.ewiseadd(vstack_helper.kronecker(front))
    return result


def _transform_front_part(front_part: cb.Matrix, constr_states_num: int) -> cb.Matrix:
    transformed_front_part = cb.Matrix.empty(front_part.shape)
    # Perform the transformation by rows
    for i, j in front_part.to_list():
        # If the element is from the constraint part
        if j < constr_states_num:
            non_zero_row_right = front_part[i: i + 1, constr_states_num:]
            # If the right part contains non-zero elements
            if non_zero_row_right.nvals > 0:
                # Account for separated front
                row_shift = i // constr_states_num * constr_states_num
                # Mark the row in the left part
                transformed_front_part[row_shift + j, j] = True
                # Update right part of the row
                for _, r_j in non_zero_row_right:
                    transformed_front_part[
                        row_shift + j, constr_states_num + r_j
                    ] = True
    return transformed_front_part



In [ ]:
import networkx as nx
import pyformlang.regular_expression as re
from typing import TypeVar

from project.automata_utils import graph_to_nfa
from project.automata_utils import regex_to_min_dfa

_NodeType = TypeVar("_NodeType")


def rpq_by_tensor_cuda(
    graph: nx.Graph,
    query: str | re.Regex,
    starts: set[_NodeType] | None = None,
    finals: set[_NodeType] | None = None,
) -> set[tuple[_NodeType, _NodeType]]:
    # Create boolean decompositions for the graph and the query
    graph_decomp = BoolDecompositionCuda.from_nfa(graph_to_nfa(graph, starts, finals))
    query_decomp = BoolDecompositionCuda.from_nfa(regex_to_min_dfa(query))

    # Intersection of decompositions gives intersection of languages
    intersection = graph_decomp.intersect(query_decomp)
    # Transitive closure helps determine reachability
    transitive_closure_indices = intersection.transitive_closure_any_symbol()

    # Two nodes satisfy the query if one is the beginning of a path (i.e. a word) and
    # the other is its end
    results = set()
    for n_from_i, n_to_i in zip(*transitive_closure_indices):
        n_from = intersection.states[n_from_i]
        n_to = intersection.states[n_to_i]
        if n_from.is_start and n_to.is_final:
            beg_graph_node = n_from.data[0]
            end_graph_node = n_to.data[0]
            results.add((beg_graph_node, end_graph_node))
    return results


def rpq_by_bfs_cuda(
    graph: nx.Graph,
    query: str | re.Regex,
    starts: set[_NodeType] | None = None,
    finals: set[_NodeType] | None = None,
    mode: BfsMode = BfsMode.FIND_COMMON_REACHABLE_SET,
) -> set[_NodeType] | set[tuple[_NodeType, _NodeType]]:
    graph_decomp = BoolDecompositionCuda.from_nfa(graph_to_nfa(graph, starts, finals))
    query_decomp = BoolDecompositionCuda.from_nfa(regex_to_min_dfa(query))

    result_indices = graph_decomp.constrained_bfs(
        query_decomp, separated=mode == BfsMode.FIND_REACHABLE_FOR_EACH_START
    )

    match mode:
        case BfsMode.FIND_COMMON_REACHABLE_SET:
            return {graph_decomp.states[i].data for i in result_indices}
        case BfsMode.FIND_REACHABLE_FOR_EACH_START:
            return {
                (graph_decomp.states[i].data, graph_decomp.states[j].data)
                for i, j in result_indices
            }



## Цель работы

Целью данной работы является анализ производительности представленных решений задачи достижимости в графе с регулярными ограничениями в трёх её постановках, описанных выше, а именно, ответ на следующие три вопроса:

1. При каких условиях использование `pycubool` даёт выигрыш в производительности по сравнению с `scipy.sparse`? Для каждой из трёх постановок задачи.
2. При использовании `pycubool`, начиная с какого размера стартового множества выгоднее решать задачу для всех пар и выбирать нужные?
3. При использовании `pycubool`, насколько решение задачи во второй её постановке быстрее решения в третьей при одинаковых начальных условиях?

## Условия эксперимента

В данной секции описывается каким образом проводился эксперимент, соответствующий цели данной работы.

### Характеристики оборудование

Оборудование, на котором проводились замеры, обладает следующими характеристиками:

In [ ]:
# @formatter:off

! echo '----- OS  -----'
! hostnamectl | grep -E 'Operating System|Kernel'

! echo '----- CPU -----'
! lscpu | grep -E 'Architecture|Model name|Thread\(s\)|Core\(s\)|Socket\(s\)|MHz|cache'

! echo '----- GPU -----'
! nvidia-smi -L
! nvidia-smi | grep CUDA

! echo '----- RAM -----'
! free -m

# @formatter:on

### Описание использованного набора данных

Для проведения экспериментов необходимо было выбрать графы для поиска путей и регулярные выражения для задания ограничений.

#### Выбор графов

Эксперимент проводился на следующих десяти графах из [датасета `CFPQ_Data`](https://jetbrains-research.github.io/CFPQ_Data/dataset/index.html):

- `skos`, `atom`, `bzip`, `pr`, `ls`, `pathways`, `enzyme`, `apache` --- как репрезентативные по количеству вершин и рёбер относительно всего датасета
- `go_hierarchy`, `geospecies` --- как имеющие сравнительно высокое отношение числа рёбер к числу вершин

Графы с числом вершин или рёбер, приближающимся к трём миллионам и выше, решено было не рассматривать, так как время их обработки оказалось достаточно велико.

In [ ]:
from project.graph_utils import load_graph_from_cfpq_data

GRAPHS = []


def init_graphs():
    for name in [
        "skos",
        "atom",
        "bzip",
        "pr",
        "ls",
        "pathways",
        "enzyme",
        "go_hierarchy",
        "apache",
        "geospecies"
    ]:
        graph = load_graph_from_cfpq_data(name)
        graph.name = name
        GRAPHS.append(graph)


init_graphs()

Ниже приведена статистика по выбранным графам.

In [ ]:
import pandas as pd
from collections import Counter
from project.graph_utils import get_graph_info


def get_most_common(source: list[str], n: int) -> list[tuple[str, int]]:
    return sorted(Counter(source).most_common(n), key=lambda x: (x[1], x[0]), reverse=True)


def graph_stats() -> pd.DataFrame:
    stats = []

    for graph in GRAPHS:
        info = get_graph_info(graph)
        stats.append([
            graph.name,
            info.nodes_num,
            info.edges_num,
            info.edges_num / info.nodes_num,
            get_most_common(info.labels, 4),
        ])

    return pd.DataFrame(
        sorted(stats, key=lambda st: st[2]),
        columns=["Name", "Nodes", "Edges", "Edges per node", "Labels (most common -> least common)"],
    )


pd.set_option("display.max_colwidth", None)
graph_stats()

#### Выбор регулярных выражений

Для каждого графа создавалось по четыре регулярных выражения следующих видов:

1. $R_1 = l_1 \ | \ l_2$ --- одно из наиболее простых выражений.
2. $R_2 = (l_1 \ | \ l_2) \ (l_3 \ | \ l_4)$ --- более сложное выражение.
3. $R_3 = l_1 \ l_2 \ l_3 \ l_4$ --- выражение со сравнительно большим числом состояний и малым числом переходов.
4. $R_4 = (l_1 \ | \ l_2)^* \ l_3$ --- выражение со сравнительно большим числом переходов и малым числом состояний.

В качестве меток $l_1, l_2, l_3, l_4$ выбирались четыре наиболее часто встречающиеся в конкретном графе метки в порядке убывания количества рёбер с ними (а если некоторые метки появляются на одинаковом количестве рёбер, то они сортируются в обратном алфавитном порядке). Если в графе менее четырёх меток, то существующие метки брались повторно в том же порядке необходимое количество раз. То есть, если граф обладает метками `ab` и `ac` на трёх рёбрах и `b` --- на одном, то $l_1 =$`ac`, $l_2 =$`ab`, $l_3 =$`b`, $l_4 =$`ac`.

Далее представлен код для генерации описанных регулярных выражений.

In [ ]:
import pyformlang.finite_automaton as fa

MAX_REGEX_LABELS_NUM = 4


def label_to_regexp(l: str) -> re.Regex:
    re0 = re.Regex("")
    re0.head = fa.Symbol(l)
    return re0


def r1(l1: str, l2: str) -> re.Regex:
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    return re1.union(re2)


def r2(l1: str, l2: str, l3: str, l4: str) -> re.Regex:
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    re12 = re1.union(re2)

    re3 = label_to_regexp(l3)
    re4 = label_to_regexp(l4)
    re34 = re3.union(re4)

    return re12.concatenate(re34)


def r3(l1: str, l2: str, l3: str, l4: str) -> re.Regex:
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    re3 = label_to_regexp(l3)
    re4 = label_to_regexp(l4)
    return re1.concatenate(re2).concatenate(re3).concatenate(re4)


def r4(l1: str, l2: str, l3: str, l4: str) -> re.Regex:
    re1 = label_to_regexp(l1)
    re2 = label_to_regexp(l2)
    re3 = label_to_regexp(l3)
    re4 = label_to_regexp(l4)
    re1234 = re1.union(re2).union(re3).union(re4)
    re1234 = re1234.kleene_star()

    return re1234.concatenate(re3).concatenate(re4)


def create_regexes_for_graph(graph: nx.Graph) -> list[re.Regex]:
    info = get_graph_info(graph)
    labels = get_most_common(info.labels, MAX_REGEX_LABELS_NUM)
    while len(labels) < MAX_REGEX_LABELS_NUM:
        labels += labels
    labels = labels[:MAX_REGEX_LABELS_NUM]

    return [
        r1(labels[0], labels[1]),
        r2(*labels),
        r3(*labels),
        r4(*labels)
    ]

Также ниже приведены характеристики данных регулярных выражений как конечных автоматов, к которым они будут сводиться в исследуемых алгоритмах.

In [ ]:
def regex_stats() -> pd.DataFrame:
    stats = []

    for name, r in [
        ("r1", r1("a", "b")),
        ("r2", r2("a", "b", "c", "d")),
        ("r3", r3("a", "b", "c", "d")),
        ("r4", r4("a", "b", "c", "d")),
    ]:
        dfa = regex_to_min_dfa(r)
        stats.append([
            name,
            len(dfa.symbols),
            len(dfa.states),
            dfa.get_number_transitions(),
            dfa.get_number_transitions() / len(dfa.states),
        ])

    return pd.DataFrame(stats, columns=["Name", "Symbols", "States", "Transitions", "Transitions per state"])


regex_stats()


#### Выбор множеств стартовых и финальных вершин

Стоит отметить, что множество стартовых вершин может заметно и нетривиально повлиять на быстродействие алгоритмов, решающих задачу в постановках 2 и 3, так как если из выбранных вершин достижима малая область графа, то обход завершится быстрее. Для алгоритма в постановке задачи под номером 3 число стартовых вершин также определяет размер матрицы-фронта. Для обеспечения воспроизводимости результатов эксперимента множества стартовых вершин генерировалось при помощи функций [`generate_multiple_source`](https://jetbrains-research.github.io/CFPQ_Data/reference/graphs/generated/cfpq_data.graphs.utils.multiple_source_utils.html#cfpq_data.graphs.utils.multiple_source_utils.generate_multiple_source) и [`generate_multiple_source_percent`](https://jetbrains-research.github.io/CFPQ_Data/reference/graphs/generated/cfpq_data.graphs.utils.multiple_source_utils.html#cfpq_data.graphs.utils.multiple_source_utils.generate_multiple_source_percent) из библиотеки `CFPQ_Data` со значением параметра `seed`, равным `42`. Так как для вопроса 2 из цели данной работы требуется изучить влияние размера множества стартовых вершин, то значение параметра `set_size` определяется конкретным экспериментом.

Напротив, множество финальных вершин влияет на быстродействие всех трёх алгоритмов тривиально и лишь своим размером: во всех случаях на завершающем этапе достигнутые вершины проверяются на принадлежность к множеству финальных вершин, что происходит через поиск по множеству. Поэтому для простоты в качестве множества финальных вершин выбиралось всё множество вершин графа.

In [ ]:
# Seed to generate the set of start nodes
START_SET_SEED = 42

### Проводимые замеры

Для ответа на каждый из поставленных вопросов нужно провести соответствующие замеры.

#### Замеры для вопроса 1 (о выгоде от использования `pycubool`)

Для выяснения того, при каких условиях использование `pycubool` даёт выигрыш в производительности против использования `scipy.sparse`, нужно выявить основные характеристики матриц, которые влияют на скорость выполнения данными библиотеками матричных операций.

Естественно предположить, что это размер и степень разреженности (мера того, насколько матрица заполнена непустыми элементами): чем больше каждая из этих величин, тем больше нетривиальных операций нужно провести. Среди параметров графа и регулярного выражения на эти величины влияют, соответственно, число вершин (или же состояний конечного автомата, в случае регулярного выражения) и отношение числа рёбер (или же переходов) к числу вершин (или же состояний).

Таким образом, для каждого из трёх алгоритмов следует для реализаций на `scipy.sparse` и `pycubool` замерить скорость их работы на всевозможных комбинациях выбранных графов и регулярных выражений с фиксированными стартовыми множествами и проследить зависимость разницы данных скоростей от указанных параметров.

#### Замеры для вопроса 2 (о влиянии размера стартового множества на выгоду использования решений первой и третьей формулировок)

Чтобы понять, при каких размерах множества стартовых вершинах, используя `pycubool`, эффективнее решать задачу в её первой постановке (то есть искать достижимые вершины среди всех пар вершин, а затем отбирать нужные), а при каких --- в третьей (то есть искать достижимые вершины лишь от стартовых вершин), требуется замерить скорость работы данных решений, реализованных на `pycubool`, при разных размерах стартового множества вершин.

Таким образом, следует замерить скорость работы реализаций решения первой и третьей постановок задачи на `pycubool` на всевозможных комбинациях выбранных графов и регулярных выражений с различными размерами стартового множества и проследить, какая из реализаций будет показывать лучший результат в зависимости от взятых размеров.

#### Замеры для вопроса 3 (о сравнении быстродействия решений второй и третьей формулировок)

Чтобы выяснить, насколько решение задачи на `pycubool` в её второй формулировке (поиск для всего стартового множества вместе) быстрее решения в третьей (поиск для каждой стартовой вершины отдельно) при равных начальных условиях нужно замерить время их работы на одинаковых графах и регулярных запросах с равными стартовыми множествами.

Таким образом, следует замерить скорость работы реализаций решения второй и третьей постановок задачи на `pycubool` на всевозможных комбинациях выбранных графов и регулярных выражений с некоторыми фиксированным стартовыми множествами и сравнить результаты.

#### Итог по необходимым замерам

Итого, для ответа на все три вопроса достаточно провести следующие замеры времени выполнения:

- на всевозможных комбинациях выбранных графов и регулярных выражений с фиксированными стартовыми множествами --- для всех реализаций;
- на всевозможных комбинациях выбранных графов, регулярных выражений и размеров стартовых множеств --- для реализаций первой и третьей формулировок задачи на `pycubool`.

Для небольших графов (с числом вершин не более 10000) время работы замеряется пять раз подряд, для больших (с числом вершин более 10000) --- два раза. Замеры проводится при помощи [`time.process_time`](https://docs.python.org/3/library/time.html#time.process_time), подсчитывается среднее время исполнения и стандартное отклонение через функции модуля `statistics`.

In [ ]:
# Percent of nodes to take as start in the fixed-start experiment
FIXED_START_SIZE = 50
# Sizes of start sets in varied-start experiment
VARIED_START_SIZES = [50, 100, 200, 400, 800, 1500, 3000, 6000, 10000]
# Number of consecutive times each measurement is run for small graphs
RUN_TIMES_SMALL = 5
# Number of consecutive times each measurement is run for huge graphs
RUN_TIMES_HUGE = 2


# Determines which graphs are considered small
def is_small(graph: nx.Graph) -> bool:
    return graph.number_of_edges() <= 10000

Ниже представлен код для проведения описанных замеров.

In [ ]:
from time import process_time
from statistics import fmean
from statistics import stdev
from typing import Callable
from cfpq_data import generate_multiple_source


def run_timed(f: Callable, run_times: int) -> tuple[float, float]:
    times = []
    for _ in range(run_times):
        start = process_time()
        f()
        finish = process_time()
        times.append(finish - start)

    m = fmean(times)
    sd = stdev(times, m)
    return m, sd


def run_fixed_starts():
    results = []

    for g_i, graph in enumerate(GRAPHS, start=1):
        regexes = create_regexes_for_graph(graph)
        run_times = RUN_TIMES_SMALL if is_small(graph) else RUN_TIMES_HUGE
        starts = generate_multiple_source(graph, FIXED_START_SIZE, seed=START_SET_SEED)
        print(f"########## Graph {graph.name} ({g_i} / {len(GRAPHS)}, run times = {run_times}) ##########")

        graph_results = []
        for r_i, regex in enumerate(regexes, start=1):
            print(f"========== Regex #{r_i} ==========")

            print("by_tensor", end=" ")
            by_tensor = run_timed(
                lambda: rpq_by_tensor(graph, regex, starts=starts),
                run_times=run_times,
            )

            print("by_bfs_all", end=" ")
            by_bfs_all = run_timed(
                lambda: rpq_by_bfs(graph, regex, starts=starts, mode=BfsMode.FIND_COMMON_REACHABLE_SET),
                run_times=run_times,
            )

            print("by_bfs_each", end=" ")
            by_bfs_each = run_timed(
                lambda: rpq_by_bfs(graph, regex, starts=starts, mode=BfsMode.FIND_REACHABLE_FOR_EACH_START),
                run_times=run_times,
            )

            print("by_tensor_cuda", end=" ")
            by_tensor_cuda = run_timed(
                lambda: rpq_by_tensor_cuda(graph, regex, starts=starts),
                run_times=run_times,
            )

            print("by_bfs_all_cuda", end=" ")
            by_bfs_all_cuda = run_timed(
                lambda: rpq_by_bfs_cuda(graph, regex, starts=starts, mode=BfsMode.FIND_COMMON_REACHABLE_SET),
                run_times=run_times,
            )

            print("by_bfs_each_cuda")
            by_bfs_each_cuda = run_timed(
                lambda: rpq_by_bfs_cuda(graph, regex, starts=starts, mode=BfsMode.FIND_REACHABLE_FOR_EACH_START),
                run_times=run_times,
            )

            graph_results.append(
                ((by_tensor, by_bfs_all, by_bfs_each), (by_tensor_cuda, by_bfs_all_cuda, by_bfs_each_cuda))
            )

        results.append(graph_results)

    return results


def run_varied_starts():
    results = []

    for g_i, graph in enumerate(GRAPHS, start=1):
        run_times = RUN_TIMES_SMALL if is_small(graph) else RUN_TIMES_HUGE
        regexes = create_regexes_for_graph(graph)
        print(f"########## Graph {graph.name} ({g_i} / {len(GRAPHS)}, run times = {run_times}) ##########")

        graph_results = []
        for r_i, regex in enumerate(regexes, start=1):
            print(f"========== Regex #{r_i} ==========")

            regex_results = []
            for start_size in VARIED_START_SIZES:
                if start_size > graph.number_of_nodes():
                    continue
                starts = generate_multiple_source(graph, start_size, seed=START_SET_SEED)
                print(f"********** Start size #{start_size} **********")

                print("by_tensor_cuda", end=" ")
                by_tensor_cuda = run_timed(
                    lambda: rpq_by_tensor_cuda(graph, regex, starts=starts),
                    run_times=run_times,
                )

                print("by_bfs_each_cuda")
                by_bfs_each_cuda = run_timed(
                    lambda: rpq_by_bfs_cuda(graph, regex, starts=starts, mode=BfsMode.FIND_REACHABLE_FOR_EACH_START),
                    run_times=run_times,
                )

                regex_results.append((by_tensor_cuda, by_bfs_each_cuda))

            graph_results.append(regex_results)

        results.append(graph_results)

    return results

## Результаты эксперимента

TODO

## Заключение

TODO